GOES-16: True Color Recipe
==========================
**Brian Blaylock**

**March 4, 2018**

brian.blaylock@utah.edu ([website](http://home.chpc.utah.edu/~u0553130/Brian_Blaylock/home.html))

(With Cartopy integration by Julien Chastang)

| Additional Notebooks                                                                                                                             |
| -------------------------------------------------------------------------------------------------------------------------------------------------|
| [GOES16 Fire Temperature](<https://github.com/blaylockbk/pyBKB_v3/blob/master/BB_GOES/mapping_GOES16_FireTemperature.ipynb>)                     |
| [GOES16 Day Convection](<https://github.com/blaylockbk/pyBKB_v3/blob/master/BB_GOES/mapping_GOES16_DayConvection.ipynb>)                         |
| [GOES16 Air Mass](<https://github.com/blaylockbk/pyBKB_v3/blob/master/BB_GOES/mapping_GOES16_AirMass.ipynb>)                                     |
| [GOES16 Day Cloud Phase Distinction](<https://github.com/blaylockbk/pyBKB_v3/blob/master/BB_GOES/mapping_GOES16_DayCloudPhaseDistinction.ipynb>) |
| [GOES16 Water Vapor](<https://github.com/blaylockbk/pyBKB_v3/blob/master/BB_GOES/mapping_GOES16_WaterVapor.ipynb>)                               |

This Python 3 notebook shows how to make a true color image from the GOES-16
Advanced Baseline Imager (ABI) level 2 data. We will plot the image with
matplotlib and Basemap. The image can be displayed on any map projection after
applying a transformation using pyproj. The methods shown here are stitched
together from the following useful information found online:

- [**True Color RGB Recipe**](http://cimss.ssec.wisc.edu/goes/OCLOFactSheetPDFs/ABIQuickGuide_CIMSSRGB_v2.pdf)
- [ABI Bands Quick Information Guides](https://www.goes-r.gov/education/ABI-bands-quick-info.html)
- [Open Commons Consortium](http://edc.occ-data.org/goes16/python/)
- [GeoNetCast Blog](https://geonetcast.wordpress.com/2017/07/25/geonetclass-manipulating-goes-16-data-with-python-part-vi/)
- [Proj documentation](https://proj4.org/operations/projections/geos.html?highlight=geostationary)
- [Pyproj documentation](http://jswhit.github.io/pyproj/pyproj.Proj-class.html)

True color images are an RGB composite of the following three channels:

|        --| Wavelength   | Channel | Description |
|----------|:------------:|:-------:|:-----------:|
| **Red**  | 0.64 &#181;m |    2    | Red Visible |
| **Green**| 0.86 &#181;m |    3    | Veggie Near-IR|
| **Blue** | 0.47 &#181;m |    1    | Blue Visible|

---
GOES-16: True Color Recipe
==========================

For this demo, you will need GOES-16 ABI level 2 data. You can get GOES-16 files
from NOAA's GOES archive on [Amazon
S3](https://aws.amazon.com/public-datasets/goes/). I created a [web
interface](http://home.chpc.utah.edu/~u0553130/Brian_Blaylock/cgi-bin/goes16_download.cgi?domain=C&product=ABI-L2-MCMIP&hour=0)
to easily download files from the Amazon archive. For scripted or bulk
downloads, you should use `rclone` or `AWS CLI`. You may also download files
from the [Environmental Data Commons](http://edc.occ-data.org/goes16/getdata/)
and [NOAA
CLASS](https://www.avl.class.noaa.gov/saa/products/search?sub_id=0&datatype_family=GRABIPRD&submit.x=25&submit.y=9).

This example uses the **level 2 _multiband_ formatted file for the _CONUS_
domain**
([ABI-L2-MCMIPC](http://home.chpc.utah.edu/~u0553130/Brian_Blaylock/cgi-bin/goes16_download.cgi?domain=C&product=ABI-L2-MCMIP&hour=0)). The
multiband file is easiest to work with becuase it contains all 16 channels on
the same 2 km grid. However, some channels have higher resolution. Plotting the
full resolution images will take some additional work, not described
here. Specifically, you will have to download three separate files, one for each
channel, and subsample the red channel 0.5 km grid to a 1 km grid.

I previously downloaded the following file from Amazon Web Services

    OR_ABI-L2-MCMIPC-M3_G16_s20181781922189_e20181781924562_c20181781925075.nc

    OR     - Indicates the system is operational  
    ABI    - Instrument type  
    L2     - Level 2 Data  
    MCMIP  - Multichannel Cloud and Moisture Imagery products  
    c      - CONUS file (created every 5 minutes).  
    M3     - Scan mode  
    G16    - GOES-16  
    s##### - Scan start: 4 digit year, 3 digit day of year (Julian day), hour, minute, second, tenth second  
    e##### - Scan end  
    c##### - File Creation  
    .nc    - NetCDF file extension  


In [1]:
######################################################################
# First, import the libraries we will use
# ---------------------------------------
#

import numpy as np
from datetime import datetime, timedelta
from pyproj import Proj
import xarray
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import os
import sys

In [ ]:
######################################################################
# Open the GOES-16 NetCDF File
# ----------------------------
# Using xarray, I assign the opened file to the variable C for the CONUS domain.
#
#FILE = 'OR_ABI-L2-MCMIPC-M3_G16_s20190591537134_e20190591539507_c20190591540015.nc'
#FILE = 'OR_ABI-L2-MCMIPC-M3_G16_s20182362117333_e20182362120105_c20182362120222.nc'
FILE = '/data/goes16/OR_ABI-L2-MCMIPM1-M3_G16_s20181442034329_e20181442034387_c20181442034458.nc'
FILE = '/data/goes16/OR_ABI-L2-MCMIPC-M3_G16_s20181910047239_e20181910050012_c20181910050123.nc'

C = xarray.open_dataset(FILE)

In [ ]:
######################################################################
# Date and Time Information
# ----------------------------
# Each file represents the data collected during one scan sequence for the
# domain. There are several different time stamps in this file, which are also
# found in the file's name.

# Scan's start time, converted to datetime object
scan_start = datetime.strptime(C.time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')

# Scan's end time, converted to datetime object
scan_end = datetime.strptime(C.time_coverage_end, '%Y-%m-%dT%H:%M:%S.%fZ')

# File creation time, convert to datetime object
file_created = datetime.strptime(C.date_created, '%Y-%m-%dT%H:%M:%S.%fZ')

# The 't' variable is the scan's midpoint time
# I'm not a fan of numpy datetime, so I convert it to a regular datetime object
midpoint = str(C['t'].data)[:-8]
scan_mid = datetime.strptime(midpoint, '%Y-%m-%dT%H:%M:%S.%f')

print('Scan Start    : %s' % scan_start)
print('Scan midpoint : %s' % scan_mid)
print('Scan End      : %s' % scan_end)
print('File Created  : %s' % file_created)
print('Scan Duration : %.2f minutes' % ((scan_end-scan_start).seconds/60))

In [ ]:
######################################################################
# True Color Recipe
# -----------------
#
# Color images are a Red-Green-Blue (RGB) composite of three different
# channels. We will assign the following channels as our RGB values:
#
#
# RGB values must be between 0 and 1, same as the range of values of the
# reflectance channels. A gamma correction is applied to control the brightness
# and make the image not look too dark where `corrected_value =
# value^(1/gamma)`. Most displays have a decoding gamma of 2.2
# ([source1](https://en.wikipedia.org/wiki/Gamma_correction),
# [source2](https://www.cambridgeincolour.com/tutorials/gamma-correction.htm)).
#
# The GREEN "veggie" channel on GOES-16 does not measure visible green
# light. Instead, it measures a near-infrared band sensitive to chlorophyll. We
# could use that channel in place of green, but it would make the green in our
# image appear too vibrant. Instead, we will tone-down the green channel by
# interpolating the value to simulate a natural green color.
#
# \begin{equation}
# TrueGreen = (0.48358168*RED) + (0.06038137*GREEN) + (0.45706946*BLUE)
# \end{equation}
#
# or, a simple alternative ([CIMSS Natural True
# Color](http://cimss.ssec.wisc.edu/goes/OCLOFactSheetPDFs/ABIQuickGuide_CIMSSRGB_v2.pdf)):
#
# \begin{equation}
# TrueGreen = (0.45*RED) + (0.1*GREEN) + (0.45*BLUE)
# \end{equation}
#
# The multiband formatted file we loaded is convenient becuase all the GOES
# channels are in the same NetCDF file. Next, we will assign our variables R, G,
# and B as the data for each channel.


# Confirm that each band is the wavelength we are interested in
for band in [2, 3, 1]:
    print("%s is %.2f %s" % (C['band_wavelength_C%02d' % band].long_name,
                             C['band_wavelength_C%02d' % band][0],
                             C['band_wavelength_C%02d' % band].units))

######################################################################
#

# Load the three channels into appropriate R, G, and B variables
R = C['CMI_C02'].data
G = C['CMI_C03'].data
B = C['CMI_C01'].data

######################################################################
#

# Apply range limits for each channel. RGB values must be between 0 and 1
R = np.clip(R, 0, 1)
G = np.clip(G, 0, 1)
B = np.clip(B, 0, 1)

######################################################################
#

# Apply a gamma correction to the image
gamma = 2.2
R = np.power(R, 1/gamma)
G = np.power(G, 1/gamma)
B = np.power(B, 1/gamma)

######################################################################
#

# Calculate the "True" Green
G_true = 0.45 * R + 0.1 * G + 0.45 * B
G_true = np.maximum(G_true, 0)
G_true = np.minimum(G_true, 1)

In [ ]:
######################################################################
# Simple Image
# -----------------
#
# Use `plt.imshow` to get a quick look at the channels and RGB composite we
# created.
#
# First, we plot each channel individually. The deeper the color means the
# satellite is observing more light in that channel. Clouds appear white becuase
# they reflect lots of red, green, and blue light. You will also notice that the
# land reflects a lot of "green" in the veggie channel becuase this channel is
# sensitive to the chlorophyll.

fig, ([ax1, ax2, ax3, ax4]) = plt.subplots(1, 4, figsize=(16, 3))

ax1.imshow(R, cmap='Reds', vmax=1, vmin=0)
ax1.set_title('Red', fontweight='semibold')
ax1.axis('off')

ax2.imshow(G, cmap='Greens', vmax=1, vmin=0)
ax2.set_title('Veggie', fontweight='semibold')
ax2.axis('off')

ax3.imshow(G_true, cmap='Greens', vmax=1, vmin=0)
ax3.set_title('"True" Green', fontweight='semibold')
ax3.axis('off')

ax4.imshow(B, cmap='Blues', vmax=1, vmin=0)
ax4.set_title('Blue', fontweight='semibold')
ax4.axis('off')

plt.subplots_adjust(wspace=.02)

In [ ]:
######################################################################
# The addition of the three channels results in a color image. We combine the
# three channels in a stacked array and display the image with `imshow` again.
#

# The RGB array with the raw veggie band
RGB_veggie = np.dstack([R, G, B])

# The RGB array for the true color image
RGB = np.dstack([R, G_true, B])

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,6))

# The RGB using the raw veggie band
ax1.imshow(RGB_veggie)
ax1.set_title('GOES-16 RGB Raw Veggie', fontweight='semibold', loc='left', fontsize=12);
ax1.set_title('%s' % scan_start.strftime('%d %B %Y %H:%M UTC '), loc='right');
ax1.axis('off')

# The RGB for the true color image
ax2.imshow(RGB)
ax2.set_title('GOES-16 RGB True Color', fontweight='semibold', loc='left', fontsize=12);
ax2.set_title('%s' % scan_start.strftime('%d %B %Y %H:%M UTC '), loc='right');
ax2.axis('off')

In [ ]:
######################################################################
# Plot with `Cartopy`:  Geostationary Projection
# ----------------------------------------------
# 
# The image above is not georeferenced. You can see the land and oceans, but we
# do have enough information to draw state and country boundaries. From the
# GOES-16 file, we can extract all the metadata needed to understand the map
# projection. We will use this data to plot the image on a `Basemap`. The image
# is in a [geostationary
# projection](https://proj4.org/operations/projections/geos.html?highlight=geostationary).

C['goes_imager_projection']
print('HERE')
######################################################################
#

# Satellite height
sat_h = C['goes_imager_projection'].perspective_point_height

# Satellite longitude
sat_lon = C['goes_imager_projection'].longitude_of_projection_origin

# Satellite sweep
sat_sweep = C['goes_imager_projection'].sweep_angle_axis

semi_major = C['goes_imager_projection'].semi_major_axis
semi_minor = C['goes_imager_projection'].semi_minor_axis

# The projection x and y coordinates equals the scanning angle (in radians)
# multiplied by the satellite height See details here:
# https://proj4.org/operations/projections/geos.html?highlight=geostationary
x = C['x'][:] * sat_h
y = C['y'][:] * sat_h

######################################################################
# The geostationary projection is the easiest way to plot the image on a
# map. Essentially, we are stretching the image across a map with the same
# projection and dimensions as the data.

fig = plt.figure(figsize=(15, 12))

globe = ccrs.Globe(semimajor_axis=semi_major, semiminor_axis=semi_minor)
geos = ccrs.Geostationary(central_longitude=sat_lon, 
                         satellite_height=sat_h, globe=globe)

ax = fig.add_subplot(1, 1, 1, projection=geos)
print('HERE2')
ax.imshow(np.flipud(RGB), origin='lower',
                   extent=(x.min(), x.max(), y.min(), y.max()),
                   transform=geos,
                   interpolation='nearest', vmin=162., vmax=330.)
print('HERE3')
ax.coastlines(resolution='110m', color='black', linewidth=2)
ax.add_feature(ccrs.cartopy.feature.COASTLINE)

plt.title('GOES-16 True Color', loc='left', fontweight='semibold', fontsize=15)
plt.title('%s' % scan_start.strftime('%d %B %Y %H:%M UTC '), loc='right');

In [ ]:
######################################################################
# Using other projections
# ----------------------------------------------
#
# Changing the projections with cartopy is straightforward. Here we use
# the Lambert Conformal projection to display the GOES-16 data.

fig = plt.figure(figsize=(15, 12))

lc = ccrs.PlateCarree()

ax = fig.add_subplot(1, 1, 1, projection=lc)
ax.set_extent([-100, -80, 18, 35], crs=ccrs.PlateCarree())

ax.imshow(np.flipud(RGB), origin='lower',
                   extent=(x.min(), x.max(), y.min(), y.max()),
                   transform=geos,
                   interpolation='none')
ax.coastlines(resolution='110m', color='black', linewidth=1)
#ax.add_feature(ccrs.cartopy.feature.COASTLINES)

plt.title('GOES-16 True Color', loc='left', fontweight='semibold', fontsize=15)
plt.title('%s' % scan_start.strftime('%d %B %Y %H:%M UTC '), loc='right')
gl = ax.gridlines(linewidth=2, color='gray', alpha=0.5, linestyle='--', draw_labels=True)

In [2]:
goes_ims = os.listdir('/data/goes16')
plfile = []
for fl in goes_ims:
    if 's2018161' in fl:
        plfile.append('/data/goes16/'+fl)

In [3]:
plfile

['/data/goes16/OR_ABI-L2-MCMIPC-M3_G16_s20181612227199_e20181612229572_c20181612230084.nc',
 '/data/goes16/OR_ABI-L2-MCMIPC-M3_G16_s20181612247199_e20181612249572_c20181612250090.nc',
 '/data/goes16/OR_ABI-L2-MCMIPC-M3_G16_s20181612207199_e20181612209572_c20181612210088.nc',
 '/data/goes16/OR_ABI-L2-MCMIPC-M3_G16_s20181612102199_e20181612104572_c20181612105089.nc',
 '/data/goes16/OR_ABI-L2-MCMIPC-M3_G16_s20181612242199_e20181612244572_c20181612245080.nc',
 '/data/goes16/OR_ABI-L2-MCMIPC-M3_G16_s20181612252199_e20181612254572_c20181612255090.nc',
 '/data/goes16/OR_ABI-L2-MCMIPC-M3_G16_s20181612017199_e20181612019572_c20181612020090.nc',
 '/data/goes16/OR_ABI-L2-MCMIPC-M3_G16_s20181612147199_e20181612149572_c20181612150085.nc',
 '/data/goes16/OR_ABI-L2-MCMIPC-M3_G16_s20181612257199_e20181612259572_c20181612300077.nc',
 '/data/goes16/OR_ABI-L2-MCMIPC-M3_G16_s20181612022199_e20181612024572_c20181612025087.nc',
 '/data/goes16/OR_ABI-L2-MCMIPC-M3_G16_s20181612212199_e20181612214572_c20181612

In [4]:
for FILE in plfile:
    ######################################################################
    # Open the GOES-16 NetCDF File
    # ----------------------------
    # Using xarray, I assign the opened file to the variable C for the CONUS domain.
    #
    #FILE = 'OR_ABI-L2-MCMIPC-M3_G16_s20190591537134_e20190591539507_c20190591540015.nc'
    #FILE = 'OR_ABI-L2-MCMIPC-M3_G16_s20182362117333_e20182362120105_c20182362120222.nc'
    #FILE = '/data/goes16/OR_ABI-L2-MCMIPM1-M3_G16_s20181442034329_e20181442034387_c20181442034458.nc'
    #FILE = '/data/goes16/OR_ABI-L2-MCMIPC-M3_G16_s20181910047239_e20181910050012_c20181910050123.nc'

    C = xarray.open_dataset(FILE)
    ######################################################################
    # Date and Time Information
    # ----------------------------
    # Each file represents the data collected during one scan sequence for the
    # domain. There are several different time stamps in this file, which are also
    # found in the file's name.

    # Scan's start time, converted to datetime object
    scan_start = datetime.strptime(C.time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')

    # Scan's end time, converted to datetime object
    scan_end = datetime.strptime(C.time_coverage_end, '%Y-%m-%dT%H:%M:%S.%fZ')

    # File creation time, convert to datetime object
    file_created = datetime.strptime(C.date_created, '%Y-%m-%dT%H:%M:%S.%fZ')

    # The 't' variable is the scan's midpoint time
    # I'm not a fan of numpy datetime, so I convert it to a regular datetime object
    midpoint = str(C['t'].data)[:-8]
    scan_mid = datetime.strptime(midpoint, '%Y-%m-%dT%H:%M:%S.%f')

    print('Scan Start    : %s' % scan_start)
    print('Scan midpoint : %s' % scan_mid)
    print('Scan End      : %s' % scan_end)
    print('File Created  : %s' % file_created)
    print('Scan Duration : %.2f minutes' % ((scan_end-scan_start).seconds/60))
    ######################################################################
    # True Color Recipe
    # -----------------
    #
    # Color images are a Red-Green-Blue (RGB) composite of three different
    # channels. We will assign the following channels as our RGB values:
    #
    #
    # RGB values must be between 0 and 1, same as the range of values of the
    # reflectance channels. A gamma correction is applied to control the brightness
    # and make the image not look too dark where `corrected_value =
    # value^(1/gamma)`. Most displays have a decoding gamma of 2.2
    # ([source1](https://en.wikipedia.org/wiki/Gamma_correction),
    # [source2](https://www.cambridgeincolour.com/tutorials/gamma-correction.htm)).
    #
    # The GREEN "veggie" channel on GOES-16 does not measure visible green
    # light. Instead, it measures a near-infrared band sensitive to chlorophyll. We
    # could use that channel in place of green, but it would make the green in our
    # image appear too vibrant. Instead, we will tone-down the green channel by
    # interpolating the value to simulate a natural green color.
    #
    # \begin{equation}
    # TrueGreen = (0.48358168*RED) + (0.06038137*GREEN) + (0.45706946*BLUE)
    # \end{equation}
    #
    # or, a simple alternative ([CIMSS Natural True
    # Color](http://cimss.ssec.wisc.edu/goes/OCLOFactSheetPDFs/ABIQuickGuide_CIMSSRGB_v2.pdf)):
    #
    # \begin{equation}
    # TrueGreen = (0.45*RED) + (0.1*GREEN) + (0.45*BLUE)
    # \end{equation}
    #
    # The multiband formatted file we loaded is convenient becuase all the GOES
    # channels are in the same NetCDF file. Next, we will assign our variables R, G,
    # and B as the data for each channel.


    # Confirm that each band is the wavelength we are interested in
    for band in [2, 3, 1]:
        print("%s is %.2f %s" % (C['band_wavelength_C%02d' % band].long_name,
                                 C['band_wavelength_C%02d' % band][0],
                                 C['band_wavelength_C%02d' % band].units))

    ######################################################################
    #

    # Load the three channels into appropriate R, G, and B variables
    R = C['CMI_C02'].data
    G = C['CMI_C03'].data
    B = C['CMI_C01'].data

    ######################################################################
    #

    # Apply range limits for each channel. RGB values must be between 0 and 1
    R = np.clip(R, 0, 1)
    G = np.clip(G, 0, 1)
    B = np.clip(B, 0, 1)

    ######################################################################
    #

    # Apply a gamma correction to the image
    gamma = 2.2
    R = np.power(R, 1/gamma)
    G = np.power(G, 1/gamma)
    B = np.power(B, 1/gamma)

    ######################################################################
    #

    # Calculate the "True" Green
    G_true = 0.45 * R + 0.1 * G + 0.45 * B
    G_true = np.maximum(G_true, 0)
    G_true = np.minimum(G_true, 1)
    ######################################################################
    # Simple Image
    # -----------------
    #
    # Use `plt.imshow` to get a quick look at the channels and RGB composite we
    # created.
    #
    # First, we plot each channel individually. The deeper the color means the
    # satellite is observing more light in that channel. Clouds appear white becuase
    # they reflect lots of red, green, and blue light. You will also notice that the
    # land reflects a lot of "green" in the veggie channel becuase this channel is
    # sensitive to the chlorophyll.
    
    # The RGB array with the raw veggie band
    RGB_veggie = np.dstack([R, G, B])

    # The RGB array for the true color image
    RGB = np.dstack([R, G_true, B])
    C['goes_imager_projection']
    print('HERE')
    ######################################################################
    #

    # Satellite height
    sat_h = C['goes_imager_projection'].perspective_point_height

    # Satellite longitude
    sat_lon = C['goes_imager_projection'].longitude_of_projection_origin

    # Satellite sweep
    sat_sweep = C['goes_imager_projection'].sweep_angle_axis

    semi_major = C['goes_imager_projection'].semi_major_axis
    semi_minor = C['goes_imager_projection'].semi_minor_axis

    # The projection x and y coordinates equals the scanning angle (in radians)
    # multiplied by the satellite height See details here:
    # https://proj4.org/operations/projections/geos.html?highlight=geostationary
    x = C['x'][:] * sat_h
    y = C['y'][:] * sat_h
    globe = ccrs.Globe(semimajor_axis=semi_major, semiminor_axis=semi_minor)
    geos = ccrs.Geostationary(central_longitude=sat_lon, 
                             satellite_height=sat_h, globe=globe)

    ######################################################################
    # Using other projections
    # ----------------------------------------------
    #
    # Changing the projections with cartopy is straightforward. Here we use
    # the Lambert Conformal projection to display the GOES-16 data.

    fig = plt.figure(figsize=(15, 12))

    lc = ccrs.PlateCarree()

    ax = fig.add_subplot(1, 1, 1, projection=lc)
    ax.set_extent([-97.5, -95, 27, 30], crs=ccrs.PlateCarree())

    ax.imshow(np.flipud(RGB), origin='lower',
                       extent=(x.min(), x.max(), y.min(), y.max()),
                       transform=geos,
                       interpolation='none')
    ax.coastlines(resolution='110m', color='black', linewidth=1)
    #ax.add_feature(ccrs.cartopy.feature.COASTLINES)

    plt.title('GOES-16 True Color', loc='left', fontweight='semibold', fontsize=15)
    plt.title('%s' % scan_start.strftime('%d %B %Y %H:%M UTC '), loc='right')
    gl = ax.gridlines(linewidth=2, color='gray', alpha=0.5, linestyle='--', draw_labels=True)
    plt.savefig('%s' % scan_start.strftime('/data/goes16/goes_out_%Y%m%d%H%M.jpg'))
    plt.close(fig )

Scan Start    : 2018-06-10 22:27:19.900000
Scan midpoint : 2018-06-10 22:28:38.500000
Scan End      : 2018-06-10 22:29:57.200000
File Created  : 2018-06-10 22:30:08.400000
Scan Duration : 2.62 minutes
ABI band 2 central wavelength is 0.64 um
ABI band 3 central wavelength is 0.87 um
ABI band 1 central wavelength is 0.47 um
HERE
Scan Start    : 2018-06-10 22:47:19.900000
Scan midpoint : 2018-06-10 22:48:38.600000
Scan End      : 2018-06-10 22:49:57.200000
File Created  : 2018-06-10 22:50:09
Scan Duration : 2.62 minutes
ABI band 2 central wavelength is 0.64 um
ABI band 3 central wavelength is 0.87 um
ABI band 1 central wavelength is 0.47 um
HERE
Scan Start    : 2018-06-10 22:07:19.900000
Scan midpoint : 2018-06-10 22:08:38.500000
Scan End      : 2018-06-10 22:09:57.200000
File Created  : 2018-06-10 22:10:08.800000
Scan Duration : 2.62 minutes
ABI band 2 central wavelength is 0.64 um
ABI band 3 central wavelength is 0.87 um
ABI band 1 central wavelength is 0.47 um
HERE
Scan Start    : 2018

HERE
Scan Start    : 2018-06-10 21:12:19.900000
Scan midpoint : 2018-06-10 21:13:38.500000
Scan End      : 2018-06-10 21:14:57.200000
File Created  : 2018-06-10 21:15:08
Scan Duration : 2.62 minutes
ABI band 2 central wavelength is 0.64 um
ABI band 3 central wavelength is 0.87 um
ABI band 1 central wavelength is 0.47 um
HERE
Scan Start    : 2018-06-10 20:57:19.900000
Scan midpoint : 2018-06-10 20:58:38.500000
Scan End      : 2018-06-10 20:59:57.200000
File Created  : 2018-06-10 21:00:07.800000
Scan Duration : 2.62 minutes
ABI band 2 central wavelength is 0.64 um
ABI band 3 central wavelength is 0.87 um
ABI band 1 central wavelength is 0.47 um
HERE
Scan Start    : 2018-06-10 20:42:19.900000
Scan midpoint : 2018-06-10 20:43:38.500000
Scan End      : 2018-06-10 20:44:57.200000
File Created  : 2018-06-10 20:45:08
Scan Duration : 2.62 minutes
ABI band 2 central wavelength is 0.64 um
ABI band 3 central wavelength is 0.87 um
ABI band 1 central wavelength is 0.47 um
HERE
Scan Start    : 2018-0

In [ ]:
'%s' % scan_start.strftime('/data/goes16/goes_out_%Y%m%d%H%M.png')

In [ ]:
cond